In [ ]:
### Set up 
!pip install torch torchvision gym

In [ ]:
import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

from ipywidgets import FloatProgress
from IPython.display import display

import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams.update({'font.size': 12})
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    print("Using", torch.cuda.device_count() ,"GPUs.")
    gpu_ids = list(range(torch.cuda.device_count()))
else:
    print("Using CPU.")
    gpu_ids = []

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
### Prioritized Buffer 

In [ ]:
class PrioritizedBuffer(object):
    def __init__(self, capacity, prob_alpha= 0.6):
        self.prob_alpha = prob_alpha
        self.capacity   = capacity
        self.buffer     = []
        self.pos        = 0
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        self.states_count = {0:0, 1:0, 'frac': 0}
        self.states_loss = {0:0, 1:0, 'frac': 0}
        self.sample_counts = np.zeros(capacity)

    def push(self, meta_state, action, reward, meta_next_state, done):
        state = meta_state[0]
        state_env = meta_state[1]
        next_state = meta_next_state[0]

        assert state.ndim == next_state.ndim
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)

        max_prio = self.priorities.max() if self.buffer else 1.0

        if len(self.buffer) < self.capacity:
            self.buffer.append((state, action, reward, next_state, done, int(state_env)))
        else:
            self.buffer[self.pos] = (state, action, reward, next_state, done, int(state_env))

        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        probs  = prios ** self.prob_alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]
        
        self.sample_counts[indices] += 1

        total    = len(self.buffer)
        weights  = (total * probs[indices]) ** (-beta)
        weights /= (weights.max()) # FIXIT: What if the max is zero?
        weights  = np.array(weights, dtype=np.float32)

        batch       = list(zip(*samples))
        states      = np.concatenate(batch[0])
        actions     = batch[1]
        rewards     = batch[2]
        next_states = np.concatenate(batch[3])
        dones       = batch[4]
        state_envs = list(batch[5])

        # increment states count
        self.states_count[0] += len(state_envs) - sum(state_envs) # states.shape[0] - np.sum(states[:,-1])
        self.states_count[1] += sum(state_envs)  # np.sum(states[:,-1])

        return states, actions, rewards, next_states, dones, indices, weights, state_envs

    def update_priorities(self, batch_indices, batch_priorities):
        for idx, prio in zip(batch_indices, batch_priorities):
            self.priorities[idx] = prio
            
    def get_average_weight_by_env(self):
        # may be worth switching to np array buffer for efficiency
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        probs  = prios ** self.prob_alpha
        probs /= probs.sum()
        
        noisy_weight_sum = 0
        std_weight_sum = 0
        noisy_count = 0
        for i in range(len(self.buffer)):
            state = self.buffer[i][0]
            is_noisy_state = (state.squeeze()[-1] == 1)
            if is_noisy_state:
                noisy_weight_sum += probs[i]
                noisy_count += 1
            else:
                std_weight_sum += probs[i]
                
        return {'std_avg': std_weight_sum / (len(self.buffer) - noisy_count),
                'noisy_avg': noisy_weight_sum / noisy_count,
                'noisy_count': noisy_count,
                'std_count': len(self.buffer) - noisy_count}
            

    def __len__(self):
        return len(self.buffer)

In [ ]:
beta_start = 0.4
beta_frames = 1000
beta_by_frame = lambda frame_idx: min(1.0, beta_start + frame_idx * (1.0 - beta_start) / beta_frames)

In [ ]:
env_id = "CartPole-v0"
env = gym.make(env_id)
val_env = gym.make(env_id)
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

In [ ]:
### DQN 

In [ ]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
        )
        self.fc = nn.Linear(128, env.action_space.n)

    def forward(self, x, return_latent = 'last'):
        """Args:
        	 return_latent: 'last': return last hidden vector
           								'state': return the state
        """
        hidden = self.layers(x)
        out = self.fc(F.relu(hidden))
        if return_latent == "state":
          	return out, state
        return out, hidden 

    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
            q_value,_  = self.forward(state)
            action  = q_value.max(1)[1].data[0]
            action = int(action)
        else:
            action = random.randrange(env.action_space.n)
        return action

In [ ]:
def update_target(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())

In [ ]:
### TD Loss 

In [ ]:
class TDLoss():

    def __init__(self,batch_size = 32, theta = 1, mode = "dot", exp = False, meg_norm = False, average_q_values = False):
        """Args:
         mode: "dot" or "euc", the distance function for averaging
         theta: power of weights (see paper) """
        super(TDLoss, self).__init__()
        self.batch_size = batch_size
        self.theta = theta
        self.mode = mode
        self.exp = exp
        self.meg_norm = meg_norm
        self.hidden = "hidden"
        self.average_q_values = average_q_values
        self.gamma = 0.99

    def hidden_weights(self, h):
        # TODO: rename normalized norm for clarity, implement normalized and unnormalized version (unnormalized
        # seems to be standard in CS224n coverage of attention)
        if self.meg_norm:
          # Meg Norm: w[i,j] = h[i].h[j]/|h[i]||h[j]|
          h = h / torch.reshape(torch.norm(torch.mul(h, h), dim = 1, p = 2), (-1,1))
        
        weights = torch.mm(h,torch.transpose(h, 0, 1))
        # Make weights positive and normalize with softmax (maps (-inf, inf) to [0,1] while maintaining ordering)
        weights = torch.nn.functional.softmax(weights, dim=1)**self.theta
        return weights

    def hidden_mean(self, h, tensor):
        if self.exp:
            tensor = torch.exp(tensor)
        tensor = torch.reshape(tensor,(-1,1))

        if self.mode == "dot":
            weights = self.hidden_weights(h)
        elif self.mode == "euc":
            # TODO: Implement euclidean_weights
            raise Exception("euclidean_weights not implemented")

        output = torch.mm(weights, tensor)
        output = output.squeeze(1)
        if self.exp:
            return torch.log(output * self.batch_size)
        return output * self.batch_size

    def compute_td_loss(self, cur_model, tar_model, beta, replay_buffer, optimizer):
        state, action, reward, next_state, done, indices, weights, state_envs = replay_buffer.sample(self.batch_size, beta)

        state      = Variable(torch.FloatTensor(np.float32(state)))
        next_state = Variable(torch.FloatTensor(np.float32(next_state)))
        action     = Variable(torch.LongTensor(action))
        reward     = Variable(torch.FloatTensor(reward))
        done       = Variable(torch.FloatTensor(done))
        weights    = Variable(torch.FloatTensor(weights))
        
        # predict q value and store hidden state if averaging q values
        if self.average_q_values:
            q_values, hiddens = cur_model.forward(state, return_latent = "last")
        else:
            q_values, hiddens = cur_model.forward(state, return_latent = None)
        next_q_values, _ = tar_model(next_state)

        q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        next_q_value     = next_q_values.max(1)[0]
        expected_q_value = reward + self.gamma * next_q_value * (1 - done)

        loss  = (q_value - expected_q_value.detach()).pow(2) * weights
        loss  = loss.mean()

        if self.average_q_values:
            # TODO: computing average over only sampled hidden states is limiting. Ideal case: compute over
            # entire buffer each time. More realistic, have buffer that stores hidden state (limits size of buffer,
            # risk of stale entries if large). Interesting spot for experimentation
            q_value = self.hidden_mean(hiddens, q_value)
            # should not be averaging expected q value--the goal of the algorithm is to have each individual
            # prediction be close to the averaged prediction
            # possible issue with task as it is defined now: it is beneficial to average noisy tasks with non-noisy
            # (though not the reverse)
            # TODO: investigate hidden state similarity
#              expected_q_value = self.hidden_mean(hiddens, expected_q_value)

        prios = (q_value - expected_q_value.detach()).pow(2) * weights + 1e-5

        optimizer.zero_grad()
        loss.backward()
        replay_buffer.update_priorities(indices, prios.data.cpu().numpy())
        optimizer.step()

        return loss

In [ ]:
### Test agent on standard or noisy env 

In [ ]:
# Test agent on standard or noisy env
# Resulting episode reward stored in XXX_val_rewards where XXX is standard or noisy
def test(noisyGame, eps, num_val_trials, current_model):
    rewards = []
    for i in range(num_val_trials):
        epsilon = 0
        episode_reward = 0
        state = val_env.reset()
        state = np.append(state, float(noisyGame))
        with torch.no_grad():
            while True:
                original_action = current_model.act(state, epsilon)

                if original_action != int(original_action):
                    original_action = original_action.numpy()[0]

                actual_action = original_action
                next_state, reward, done, _ = val_env.step(actual_action)
                next_state = np.append(next_state, float(noisyGame))

                if noisyGame:
                    reward += random.uniform(-1., 1.)

                state = next_state
                episode_reward += reward

                if done:
                    rewards.append(episode_reward)
                    break
    return np.mean(rewards)

In [ ]:
### Train agent 

In [ ]:
def train(variance, theta, exp, average_q_values, meg_norm, hardcoded, invert_actions = False, num_frames = 10000, num_val_trials = 10, batch_size = 32, gamma = 0.99, num_trials = 5, USE_CUDA = False, device = "", eps = 1.):

    # Progress bar
    f = FloatProgress(min=0, max=num_frames)
    display(f)

    device = torch.device("cuda")

    """Args:"""
    losses = []
    all_rewards = []
    standard_val_rewards = []
    noisy_val_rewards = []
    states_count_ratios = []
    episode_reward = 0

    # Initialize state
    noisyGame = False
    state = env.reset()
    state = np.append(state, float(noisyGame)) # BACK IN
    meta_state = (state, float(noisyGame))

    # Initialize replay buffer, model, TD loss, and optimizers
    replay_buffer = PrioritizedBuffer(100000)
    # replay_buffer = PrioritizedBuffer(1000)
    current_model = DQN(env.observation_space.shape[0] + 1, env.action_space.n) # BACK IN
    target_model  = DQN(env.observation_space.shape[0] + 1, env.action_space.n) # BACK IN
    td_loss = TDLoss(average_q_values=average_q_values)
    optimizer = optim.Adam(current_model.parameters())
    
    # Multi GPU - Under Construction.
#     current_model = current_model.to(device)
#     target_model = target_model.to(device)

#     # Single GPU Code
    if USE_CUDA:
        current_model = nn.DataParallel(current_model, gpu_ids)
        target_model = nn.DataParallel(target_model, gpu_ids)
        current_model = current_model.cuda()
        target_model  = target_model.cuda()

    result_df = pd.DataFrame()
    power = theta
    var = variance
    all_standard_val_rewards = []
    all_proportions = []
    std_weights = []
    noisy_weights = []
    std_buffer_example_count = []
    noisy_buffer_example_count = [] 
    
    eps = .5

    for t in range(num_trials):
        print("trial number: {}".format(t))
        #print("is noisy game?", noisyGame)
        for frame_idx in range(1, num_frames + 1):
            epsilon = epsilon_by_frame(frame_idx)
            original_action = current_model.act(state, epsilon)

            # If in noisy environment, make action random with probability eps
            if noisyGame and random.uniform(0,1) < eps:
                if invert_actions:
                    actual_action = 1 - original_action # invert
                else:
                    actual_action = original_action
            else:
                actual_action = original_action

            next_state, reward, done, _ = env.step(actual_action)

            # If in noisy environment, make reward completely random
            if noisyGame:
                reward *= np.random.normal(0, var)

            next_state = np.append(next_state, float(noisyGame))
            meta_next_state = (next_state, float(noisyGame))
            replay_buffer.push(meta_state, original_action, reward, meta_next_state, done)

            meta_state = meta_next_state
            episode_reward += reward

            if done:
                noisyGame = 1-noisyGame
                state = env.reset()
                state = np.append(state, float(noisyGame))
                meta_state = (state, float(noisyGame))
                all_rewards.append(episode_reward)
                episode_reward = 0

            if len(replay_buffer) > batch_size:
                beta = beta_by_frame(frame_idx)
                loss = td_loss.compute_td_loss(current_model, target_model, beta, replay_buffer, optimizer)
                losses.append(loss.data.tolist())

            if frame_idx % 200 == 0:
                all_standard_val_rewards.append(test(False, eps, num_val_trials, current_model))
                all_proportions.append(float(replay_buffer.states_count[1]) / (float(replay_buffer.states_count[1])  + float(replay_buffer.states_count[0])))
                weight_dict = replay_buffer.get_average_weight_by_env()
                std_weights.append(weight_dict['std_avg'])
                noisy_weights.append(weight_dict['noisy_avg'])
                std_buffer_example_count.append(weight_dict['std_count'])
                noisy_buffer_example_count.append(weight_dict['noisy_count'])
                
                # Verified that standard experience replay weights noisy examples more heavily and oversamples
                # TODO: run complete experiments and check improvement with proposed replay technique
#                 print(weight_dict)
#                 print('Noisy to std weight', noisy_weights[-1]/std_weights[-1])
#                 print('Proportion of noisy selected (long run): ', all_proportions[-1])
                
            #         plot(frame_idx, all_rewards, losses, standard_val_rewards, noisy_val_rewards, states_count_ratios)

            if frame_idx % 1000 == 0:
                update_target(current_model, target_model)
                print("frame_idx", frame_idx)

            f.value += 1

    result_df['trial_num'] = list(range(50)) * num_trials # 50 = 10000 frames / 200 frams
    result_df['val_reward'] = all_standard_val_rewards
    result_df['proportion'] = all_proportions
    result_df['std_weights'] = std_weights
    result_df['noisy_weights'] = noisy_weights
    result_df['std_buffer_example_count'] = std_buffer_example_count
    result_df['noisy_buffer_example_count'] = noisy_buffer_example_count
    return result_df

In [ ]:
### Run experiments 

In [ ]:
def run_experiments(variances = np.logspace(0.1, 10, 4), thetas = np.logspace(0.1, 5, 4), exps = [False], meg_norms = [True, False], hardcodeds=[True, False], average_q_values=[False]):
    meta_dict = {}
    #pd.DataFrame(columns=['var', 'theta', 'exp', 'meg_norm', 'result_df'])
    for var in variances:
        for theta in thetas:
            for exp in exps:
                for avg_q in average_q_values:
                    for meg_norm in meg_norms:
                        for hardcoded in hardcodeds:
                            result_df = train(var, theta, exp, avg_q, meg_norm, hardcoded, invert_actions = False, num_frames = 10000, num_val_trials = 10, batch_size = 32, num_trials = 5, USE_CUDA = USE_CUDA)
                            key = [var, theta, exp, meg_norm, hardcoded]
                            meta_dict[key] = result_df
    return meta_dict

In [ ]:
# meta_dict = run_experiments(variances = np.logspace(0.1, 10, 4), 
#                             thetas = np.logspace(0.1, 5, 4), 
#                             exps = [False], 
#                             meg_norms = [True, False], 
#                             hardcodeds=[True, False])

meta_dict = run_experiments(variances = [.5], 
                            thetas = [1], 
                            exps = [False], 
                            meg_norms = [False], 
                            hardcodeds=[False],
                            average_q_values=[False])

In [ ]:
del meta_dict